In [4]:
import well_profile as wp
import numpy as np
import pandas as pd 

df = pd.read_csv('/Users/prathikprasad/Desktop/DataRepo Hook Load/LDG_main_survey.csv')
df = df.rename(columns={'MD(ftKB)': 'md', 'Incl(deg)': 'inc', 'Azm(deg)': 'azi'})
well_trajectory = wp.load(df)

In [9]:
def interpolate(md , well_trajectory):
    tvd = np.zeros(len(md))
    inc = np.zeros(len(md))
    azi = np.zeros(len(md))
    for i in range(0 , len(md)):
        pt = well_trajectory.get_point(md[i])
        tvd[i] = pt['tvd']

        try:
            inc[i] = pt['inc']
        except:
            if i > 0:
                inc[i] = inc[i-1]
            else:
                inc[i] = 0
            try:
                azi[i] = pt['azi']
            except:
                if i > 0:
                    azi[i] = azi[i-1]
                else:
                    azi[i] = 0  

    return tvd , inc , azi 



def discretize(dx , bit_depth , OD , ID):

    n_elements = int(np.ceil(bit_depth/dx))  
    l = np.full(n_elements, bit_depth / n_elements)

    J = np.pi / 32 * (np.power(OD / 12, 4) - np.power(ID / 12, 4))
    w = l * 2.67 * (np.power(OD, 2) - np.power(ID, 2))

    x = np.linspace(0, bit_depth, n_elements)

    return {'l': l, 'J': J, 'w': w, 'x': x, 'n_elements': n_elements}



def normal_force(bha, well, mu=0.3, WOB=0, TOB=0):
    # First, discretize the BHA
    bha = discretize(bha['dx'], bha['bit_depth'], bha['OD'], bha['ID'])

    
    tvd, inc, azi = interpolate(bha['x'], well)

    
    n_elements = bha['n_elements']
    Ft = np.zeros(n_elements)       # Tension
    Ft_pu = np.zeros(n_elements)    # Tension, pulling up
    Ft_so = np.zeros(n_elements)    # Tension, slacking off
    Fn = np.zeros(n_elements)       # Normal force
    M = np.zeros(n_elements)        # Torque

    # Convert azimuth and inclination to radians
    azi = azi / 180 * np.pi
    inc = inc / 180 * np.pi

    # Calculate the change in azimuth and inclination across each element
    d_azi = azi[1:] - azi[:-1]
    d_inc = inc[1:] - inc[:-1]

    for i in range(0 , n_elements):
        if i==0:
            M[n_elements-1] = 0
            Ft[n_elements-1] = 0

        else:
            fn = np.sqrt(np.power(Ft[n_elements-i]*d_azi[n_elements-1-i]*np.sin(inc[n_elements-i]) , 2) +
                         np.power(Ft[n_elements-i]*d_inc[n_elements-1-i] + bha['w'][n_elements-i]*np.sin(inc[n_elements-i]) , 2))
            dft = bha['w'][n_elements-i]*np.cos(inc[n_elements-i])
            fric = mu * fn

            dM = mu * fn * bha['OD'][n_elements-i] / 24

            Ft[n_elements-1-i] = Ft[n_elements-i] + dft
            Ft_pu[n_elements-1-i] = Ft_pu[n_elements-i]+ dft + fric
            Ft_so[n_elements-1-i] = Ft_so[n_elements-i] + dft - fric
            Fn[n_elements-1-i] = fn
            M[n_elements-1-i] = M[n_elements-i] + dM

    return {'Ft': Ft, 'Ft_pu': Ft_pu, 'Ft_so': Ft_so, 'Fn': Fn, 'M': M}




In [ ]:
def process_all(df_dict, well_trajectory, dx, OD, ID):
    results = {}
    for key, df in df_dict.items():
        md = df['md']  # assuming 'md' is a column in your data frame
        bit_depth = df['bit_depth'].max()  # calculate the maximum bit depth for each data frame

        tvd, inc, azi = interpolate(md, well_trajectory)
        well = {'tvd': tvd, 'inc': inc, 'azi': azi}
        
        bha = discretize(dx, bit_depth, OD, ID)
        result = normal_force(bha, well)
        
        results[key] = result
    return results

results = process_all(df_all_tripout_filter, well_trajectory, dx, OD, ID)